Use interactive session


In [1]:
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from pprint import pprint

from tensorflow.examples.tutorials.mnist import input_data

import numpy as np



In [2]:
mnist = input_data.read_data_sets("tmp/data/", one_hot = True)

n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500
n_classes = 10
batch_size = 100

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting tmp/data/t10k-images-idx3-ubyte.gz
Extracting tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
x = tf.placeholder('float', [None, 784])
y = tf.placeholder('float')

In [4]:
class TfAnn(object):
    
    def __init__(self):
        self.hidden_1_layer = {'weights':[],'biases':[]}
        self.hidden_2_layer = {'weights':[],'biases':[]}
        self.hidden_3_layer = {'weights':[],'biases':[]}
        self.output_layer = {'weights':[],'biases':[]}
        self.np_hidden_1_layer={"weights":[],"biases":[]}
        self.np_hidden_2_layer={"weights":[],"biases":[]}
        self.np_hidden_3_layer={"weights":[],"biases":[]}
        self.np_output_layer={"weights":[],"biases":[]}
    
    def init_empty(self,size,n_nodes_hl1,n_nodes_hl2,n_nodes_hl3,n_classes):
        self.hidden_1_layer = {'weights':tf.Variable(tf.random_normal([size, n_nodes_hl1])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}

        self.hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                          'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}

        self.hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                          'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))}

        self.output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                        'biases':tf.Variable(tf.random_normal([n_classes]))}


    def init_values(self,size,l1_weights,l2_weights,l3_weights,out_weights,l1_biases,l2_biases,l3_biases,out_biases):
        self.hidden_1_layer = {'weights':tf.Variable(l1_weights),'biases':tf.Variable(l1_biases)}
        self.hidden_2_layer = {'weights':tf.Variable(l2_weights),'biases':tf.Variable(l2_biases)}
        self.hidden_3_layer = {'weights':tf.Variable(l3_weights),'biases':tf.Variable(l3_biases)}
        self.output_layer = {'weights':tf.Variable(out_weights),'biases':tf.Variable(out_biases)}


    def create(self,data):
        l1 = tf.add(tf.matmul(data,self.hidden_1_layer['weights']), self.hidden_1_layer['biases'])
        l1 = tf.nn.relu(l1)

        l2 = tf.add(tf.matmul(l1,self.hidden_2_layer['weights']), self.hidden_2_layer['biases'])
        l2 = tf.nn.relu(l2)

        l3 = tf.add(tf.matmul(l2,self.hidden_3_layer['weights']), self.hidden_3_layer['biases'])
        l3 = tf.nn.relu(l3)

        output =  tf.add(tf.matmul(l3,self.output_layer['weights']) , self.output_layer['biases'])

        return output
    
    def extract(self):
        self.np_hidden_1_layer["weights"] = neural_network_model.hidden_1_layer["weights"].eval()
        self.np_hidden_2_layer["weights"] = neural_network_model.hidden_2_layer["weights"].eval()
        self.np_hidden_3_layer["weights"] = neural_network_model.hidden_3_layer["weights"].eval()
        self.np_output_layer["weights"] = neural_network_model.output_layer["weights"].eval()
        self.np_hidden_1_layer["biases"] = neural_network_model.hidden_1_layer["biases"].eval()
        self.np_hidden_2_layer["biases"] = neural_network_model.hidden_2_layer["biases"].eval()
        self.np_hidden_3_layer["biases"] = neural_network_model.hidden_3_layer["biases"].eval()
        self.np_output_layer["biases"] = neural_network_model.output_layer["biases"].eval()
        

In [5]:
def train_neural_network_interactive(d,neural_network_model):
    prediction = neural_network_model.create(d)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    hm_epochs = 10
    sess = tf.InteractiveSession()
    init = tf.initialize_all_variables()
    sess.run(init)
    for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c

            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)


    neural_network_model.extract()


In [6]:
def test_neural_network_interactive(d,neural_network_model):
    prediction = neural_network_model.create(d)
    hm_epochs = 10
    sess = tf.InteractiveSession()
    init = tf.initialize_all_variables()
    sess.run(init)
    correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
    print('Accuracy:',accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))

     
    y_test=(sess.run(prediction,feed_dict={
                             x: mnist.test.images
                              }))       
    
    true_class=np.argmax(mnist.test.labels,1)
    predicted_class=np.argmax(y_test,1)
    cm=confusion_matrix(predicted_class,true_class)
    pprint(cm)
    neural_network_model.extract()

In [7]:
neural_network_model = TfAnn()
neural_network_model.init_empty(784,n_nodes_hl1,n_nodes_hl2,n_nodes_hl3,n_classes)
train_neural_network_interactive(x,neural_network_model)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch 0 completed out of 10 loss: 1595294.816558838
Epoch 1 completed out of 10 loss: 371427.2099304199
Epoch 2 completed out of 10 loss: 202244.31945991516
Epoch 3 completed out of 10 loss: 116461.35738129169
Epoch 4 completed out of 10 loss: 71315.73213392496
Epoch 5 completed out of 10 loss: 45391.652174089104
Epoch 6 completed out of 10 loss: 33002.68025750469
Epoch 7 completed out of 10 loss: 27122.86840927601
Epoch 8 completed out of 10 loss: 17675.494365146646
Epoch 9 completed out of 10 loss: 17310.87047871232


In [16]:
test_neural_network_model = TfAnn()
l1_weights = neural_network_model.np_hidden_1_layer["weights"]
pprint(l1_weights.shape)
np.savetxt("Data/l1_weights.csv", l1_weights, delimiter=",")
l1_biases = neural_network_model.np_hidden_1_layer["biases"]
np.savetxt("Data/l1_biases.csv", l1_biases, delimiter=",")
l2_weights = neural_network_model.np_hidden_2_layer["weights"]
np.savetxt("Data/l2_weights.csv", l2_weights, delimiter=",")
l2_biases = neural_network_model.np_hidden_2_layer["biases"]
np.savetxt("Data/l2_biases.csv", l2_biases, delimiter=",")
l3_weights = neural_network_model.np_hidden_3_layer["weights"]
np.savetxt("Data/l3_weights.csv", l3_weights, delimiter=",")
l3_biases = neural_network_model.np_hidden_3_layer["biases"]
np.savetxt("Data/l3_biases.csv", l3_biases, delimiter=",")
out_weights = neural_network_model.np_output_layer["weights"]
np.savetxt("Data/out_weights.csv", out_weights, delimiter=",")
out_biases = neural_network_model.np_output_layer["biases"]
np.savetxt("Data/out_biases.csv", out_biases, delimiter=",")

test_neural_network_model.init_values(784,l1_weights,l2_weights,l3_weights,out_weights,l1_biases,l2_biases,l3_biases,out_biases)
test_neural_network_interactive(x,test_neural_network_model)

(784, 500)
Accuracy: 0.9503
array([[ 944,    0,    4,    0,    0,    2,    4,    0,    3,    0],
       [   0, 1112,    3,    0,    3,    2,    2,    1,    1,    8],
       [   7,    1,  980,    8,    2,    1,    4,   14,    2,    2],
       [   0,    0,    9,  952,    1,   21,    0,    4,    8,    2],
       [   0,    0,    2,    0,  900,    0,    5,    5,    5,    4],
       [   2,    1,    2,   12,    1,  826,   10,    1,    5,    4],
       [  12,    3,    4,    0,    8,    8,  918,    0,    1,    2],
       [   3,    3,    7,    8,    8,    1,    1,  965,    7,    8],
       [  10,   15,   20,   27,   12,   25,   13,    5,  936,    9],
       [   2,    0,    1,    3,   47,    6,    1,   33,    6,  970]])
